In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Read file CSV
data = pd.read_csv('ABNB_stock_data.csv')
data['Date'] = pd.to_datetime(data['Date'], dayfirst=True)
data.set_index('Date', inplace=True)

# Lagged features and rolling mean
data['Close_Lag1'] = data['Close'].shift(1)
data['5_day_MA'] = data['Close'].rolling(window=5).mean()
data = data.dropna()  # Drop rows with null values

# Data
X = data[['Close_Lag1', '5_day_MA']]
y = data['Close']

# Standardize data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.349, shuffle=False)

# Define model
model = KNeighborsRegressor()

# Define parameter grid
param_grid = {
    'n_neighbors': [5, 10, 15, 20, 25],
    'weights': ['uniform', 'distance'],
    'metric': ['minkowski', 'euclidean', 'manhattan'],
    'p': [1, 2]
}

# Define RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_grid,
    n_iter=10,
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1)

# Fit RandomizedSearchCV
random_search.fit(X_train, y_train)

# Best parameters found by RandomizedSearchCV
print(f"Best Parameters: {random_search.best_params_}")

# Best model
best_model = random_search.best_estimator_

# Predict with best model
y_pred = best_model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"R²: {r2}")